## 학습한 모델 저장하기
> 일반적인 torch.save x -> yolov5 내부 클래스를 이용   
> 최종 모델 yolov5 mAP 약 0.748

In [1]:
ROOT = 'C:/Users/5788j/Desktop/yolov5_bear-team/'

from models.common import DetectMultiBackend
import torch
import time

device = torch.device('cpu')
weight_path = ROOT + 'last_best_weight.pt'
data = ROOT +'/data/custom_data.yaml'

dnn=False
model = DetectMultiBackend(weight_path, device=device, dnn=dnn, data=data)
torch.save(model, '../yolov5_bear-team/our_last_model.pt')

Fusing layers... 
custom_YOLOv5n summary: 213 layers, 1763224 parameters, 0 gradients


## detect 함수 커스텀
> 기존 detect 함수는 쓸모없는 동작이 포함되어 추론시간이 길어져 fps down -> bear team 서비스에 맞게 커스텀 필요   

In [15]:
# import os
# import argparse
# import platform
# import sys
from pathlib import Path

import torch
# import torch.backends.cudnn as cudnn


# from models.common import DetectMultiBackend
# from utils.dataloaders import IMG_FORMATS, VID_FORMATS, LoadImages, LoadStreams
# from utils.general import (LOGGER, Profile, check_file, check_img_size, check_imshow, check_requirements, colorstr, cv2,
#                            increment_path, non_max_suppression, print_args, scale_coords, strip_optimizer, xyxy2xywh)
# from utils.plots import Annotator, colors, save_one_box
# from utils.torch_utils import select_device, smart_inference_mode
# from utils.augmentations import letterbox
from utils.general import non_max_suppression, scale_coords, xyxy2xywh
# import torch
# import time
# import json
# from collections import OrderedDict # 딕셔너리 형태인데 (key가 아닌 순서로 고려)
import numpy as np

weight_path = '../yolov5_bear-team/our_last_model.pt' # mAP 0.748
ROOT = 'C:/Users/5788j/Desktop/yolov5_bear-team/'
model = None

# @torch.no_grad()
def modified_detect(weight_path, input_img, start_token):
    '''source는 추론할 이미지 경로, save_dir은 추론 이미지 저장 경로'''
    
    global model
    weight_path = weight_path # 가중치 경로
    input_img = input_img # 이미지 경로
    device = torch.device('cpu')
    conf_thres=0.25 
    iou_thres=0.45 
    classes=None 
    max_det = 500
    hide_labels=False  # hide labels
    hide_conf=False

    # 이미지 사이즈 주기
    imgsz=(640,640)

    # load model ( modified )
    if start_token == True:
        model = torch.load(weight_path, map_location=device)
        
        
    stride, names, pt = model.stride, model.names, model.pt
    iou_thres = 0.5
    agnostic_nms = False
    max_det = 1000
    save_crop = False
    save_conf = True
    line_thickness=3

    # Run inference
    # im0는 원본 이미지(BGR), im은 전처리하여 모델에 들어갈 이미지
#     im0 = cv2.cvtColor(input_img, cv2.COLOR_RGB2BGR)
    im0 = input_img
#     im = letterbox(im0, 640, stride=32, auto=True)[0]  # padded resize
    im = im0
    im = im.transpose((2, 0, 1)) # [::-1]  # HWC to CHW, BGR to RGB
    im = np.ascontiguousarray(im)  # contiguous
#     seen, windows, dt = 0, [], (Profile(), Profile(), Profile()) # dt(delta time) -> [dt, dt, dt]
    seen = 0
# im은 전처리하여 모델에 넣을 이미지
# im0는 raw이미지 (원래 해상도의 BGR)
#     with dt[0]:
    im = torch.from_numpy(im).to(torch.float32).to(device)
#     im = im.float()#im.half() if model.fp16 else im.float()  # uint8 to fp16/32
    im /= 255.0  # 0 - 255 to 0.0 - 1.0
    if len(im.shape) == 3:
        im = im[None]  # expand for batch dim
    # Inference
#     with dt[1]:
    model.eval() # 이거는 init에 들어간다.
    with torch.no_grad(): #이거는 데코레이터로
        pred = model(im)

    # NMS
#     with dt[2]:
    pred = non_max_suppression(pred, conf_thres, iou_thres, classes, agnostic_nms, max_det=max_det)[0]
#     det = pred[0]
        # Process predictions
#     for i, det in enumerate(pred):  # per image -> i는 이미지 개수, det는 pred값
    seen += 1
#         im0_copy =  im0.copy()
# bbox unnorm을 위한 사이즈 gain            
#         gn = torch.tensor(im0_copy.shape)[[1, 0, 1, 0]]  # normalization gain whwh
    gn = torch.tensor(im0.shape)[[1, 0, 1, 0]]  # normalization gain whwh
# annotation 준비
#         annotator = Annotator(im0, line_width=line_thickness, example=str(names))        

        # bbox, confidence, cls값 저장
    inference_inform = []
    if len(pred): # 디텍션한 객체 개수
# 예측한 객체 개수 마다 bbox(det 1~4열)를 unnormalize 진행
        pred[:, :4] = scale_coords(im.shape[2:], pred[:, :4], im0.shape).round()

            # Print results
#             for c in det[:, -1].unique():
#                 n = (det[:, -1] == c).sum()  # detections per class

        for *xyxy, conf, cls in reversed(pred):
            xywh = (xyxy2xywh(torch.tensor(xyxy).view(1, 4)) / gn).view(-1)  # normalized xywh
#                 obj = Object('category': )
            inference_inform.append({
                                         'category' : int(cls),
                                         'probability' : float(conf), 
                                         'center' : xywh[:2], 
                                         'width':xywh[-2],
                                         'height':xywh[-1]
                                                            })
#                     c = int(cls)  # integer class
#                     label = None if hide_labels else (names[c] if hide_conf else f'{names[c]} {conf:.2f}')
#                     annotator.box_label(xyxy, label, color=colors(c, True))

#             # Stream results
#             im0 = annotator.result() # 예측 정보를 전부 기재한 이미지        

    return inference_inform

## Modified_Detect 함수 테스트  
> 서비스에 맞게 빠른 서비스를 위해 커스텀한 detect함수 동작, fps를 테스트한다.   

In [16]:
import glob
import cv2
import time
from tqdm import tqdm
sources = list(glob.glob('../yolov5_bear-team/validation_img' + '/*'))

result=[]
start = time.time()
for i, img in tqdm(enumerate(sources)): 
    
    if i == 0:
        start_token = True
    else:
        start_token = False
    input_img = cv2.imread(img, cv2.COLOR_BGR2RGB)
    result.append(modified_detect(weight_path, input_img, start_token))
    print(i)
    
end = time.time()
print(f'이미지 {i}장 추론 경과시간 : ', end-start)
print(f'최종 모델 fps : {round( (i+1)/(end-start), 3)}')

3it [00:00, 10.49it/s]

0
1
2



5it [00:00,  9.40it/s]

3
4


8it [00:00,  9.70it/s]

5
6
7


10it [00:01,  9.30it/s]

8
9
이미지 9장 추론 경과시간 :  1.0777127742767334
최종 모델 fps : 9.279


## testset으로 input / output 결과 확인

In [18]:
from PIL import Image, ImageDraw, ImageFont
import cv2
import random

# 랜덤 검수
i = random.randint(0, len(glob.glob('../yolov5_bear-team/validation_img' + '/*'))-1)
img_path = glob.glob('../yolov5_bear-team/validation_img' + '/*')[i]

annotation = result[i] # img_path에 있는 모든 이미지에 대해 추론한 annotation이 담긴 리스트

# 여러개여도1개의 객체만 bbox확인(test)
for ii, obj_key in enumerate(annotation):
    if ii != 0: # 객체 하나만 검수
        break

    # category == class name
    if annotation[obj_key]['category'] == 0:
        c = 'motor cycle' 
    elif annotation[obj_key]['category'] == 1:
        c = 'bicycle' 
    elif annotation[obj_key]['category'] == 2:
        c = 'kick board'
        
    raw_w, raw_h = cv2.imread(img_path).shape[:2]

    x = annotation[obj_key]['center'][0] * raw_w
    y = annotation[obj_key]['center'][1] * raw_h
    w = annotation['obj_0']['width'] * raw_w
    h = annotation['obj_0']['height'] * raw_h

# arguments
    color = (0,255,0)
    text_pos = (int(x + w/2), int(y - h/2))
    font_size = 25
#     font = ImageFont.truetype("arial.ttf", font_size) # arial.ttf 글씨체, font_size=15

    # image
    img = Image.open(img_path).convert('RGB')
    draw = ImageDraw.Draw(img)
    draw.rectangle( (int(x - w/2), int(y+h/2), int(x + w/2), int(y - h/2)), outline=color, width = 3) # 좌상단, 우하단 좌표
    draw.text(text_pos, c, color) 
img

# testing_inference(img_path, annotation)

TypeError: list indices must be integers or slices, not dict

In [19]:
torch.tensor([1, 2])

tensor([1, 2])